# Task: Predict Malignant and Benign Skin Moles

I'm starting on FastAI's online course and chose this Task for my first round of practice. If you've been through FastAI's course before this might seem firmiliar, if not take a look at some of the basics from week one. 

Because this is a balenced dataset today I'm simply going look at accuracy as the evaluation metric. This project is for educational purposes and considering that I cobbled together with some spare time on my weekend, I'm pretty happy with this. I'm not a doctor (I don't even play one on TV).

Thanks to:
* Cluadio Fanconi https://www.kaggle.com/fanconic for the Task, and also for creating a submission to compete against (His model produced an accuracy of about 92% so thats what I'm shoot for)
* The International Skin Imaging Collaboration https://www.isic-archive.com/ for the dataset. All the images are well centered and are size 224 so they made the bar for entry very reasonable.

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [ ]:
PATH = Path('/kaggle/input/skin-cancer-malignant-vs-benign/')

In [ ]:
data = ImageDataBunch.from_folder(PATH, train="train/",
#                                  valid="train/",
#                                  test="test/",
                                  valid_pct=.3,
                                  ds_tfms=get_transforms(),
                                  size=224,bs=32, 
                                  ).normalize(imagenet_stats)

In [ ]:
print(f'Classes: \n {data.classes}')

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4))

In [ ]:
Model_Path = Path('/kaggle/working/')
learn.model_dir = Model_Path
learn.save('stage-1')#checkpointing the model incase the next couple of tasks backfire

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(4,4))

# Final thoughts:
Initially this would appear to have an accuracy of 93%(at least on the validation set). Each week I challenge myself to complete one Kaggle Task and this week I didn't have enough time to do this project justice. While my results look good, I did not manage to test for overfitting (which is a serious asterisk to append to this). Later I will revisit this notebook when I'm able to retest the model against the test data instead of just a validation set.

If you have any constructive feedback feel free to message me or leave a comment. If you liked the notebook I'd appriciate the upvote. And if you would like to collaborate with me on a future Kaggle task send me a message here or on LinkedIn.

In [ ]:
learn.save('Final')